### 1数据准备与预处理

##### 1.1数据准备

In [26]:
import os
import pandas as pd
import re
import jieba
import jieba.posseg as psg


In [41]:
 
#bertopic详细资料：cx15251237804.
 
output_path = r'D:\大学作业文件\大学杂物\A比赛\2025数模\2025数模培训\第五次培训深圳杯2022A\数据\LDAsklearn_origin'
file_path = r'D:\大学作业文件\大学杂物\A比赛\2025数模\2025数模培训\第五次培训深圳杯2022A\数据\BERTopic'
os.chdir(file_path)
# data=pd.read_excel("dataset.xlsx").astype(str)#content type
# 读取CSV文件
df = pd.read_csv(r"D:\大学作业文件\大学杂物\A比赛\2025数模\2025数模培训\第五次培训深圳杯2022A\数据\caixukun.csv")

# 提取成绩列
data = df['raw_text']  # 如果列名是"正文内容"
data = pd.DataFrame(data)  # 转换为DataFrame
# data=data.astype('string')  # 确保所有元素都是字符串类型

os.chdir(output_path)
dic_file = r"D:\大学作业文件\大学杂物\A比赛\2025数模\2025数模培训\第五次培训深圳杯2022A\数据\dic.txt"
stop_file = r"D:\大学作业文件\大学杂物\A比赛\2025数模\2025数模培训\第五次培训深圳杯2022A\数据\chinesestopword.txt"

In [37]:
data[:5]  # 查看前5条数据

,raw_text
0,也许可有可无是我的存在//@葵的妈奎的妹坤的妻:[抱抱][抱抱][抱抱][抱抱]哥哥，好想你...
1,我讨厌你中间加什么字最甜？我讨厌没你...蔡徐坤//@蔡徐坤我老公鸭://@AK47-KAK...
2,月亮躲起来 连星星 都不在//@葵的妈奎的妹坤的妻:[抱抱][抱抱][抱抱][抱抱]哥哥，好...
3,我只逃避在我封闭的舞台//@葵的妈奎的妹坤的妻:[抱抱][抱抱][抱抱][抱抱]哥哥，好想你...
4,已不能重来//@葵的妈奎的妹坤的妻:[抱抱][抱抱][抱抱][抱抱]哥哥，好想你呀@蔡徐坤


In [38]:
print(type(data))  # 查看数据类型

<class 'pandas.core.frame.DataFrame'>


#### 1.2数据预处理

In [ ]:
def english_word_cut(mytext):
    stop_list = []
    try:
        with open(stop_file, encoding='utf-8') as stopword_list:
            stop_list = [line.strip() for line in stopword_list]
    except FileNotFoundError:
        print(f"Error: Stop file '{stop_file}' not found.")
    word_list = []
    # 使用正则表达式将文本分割为单词
    words = re.findall(r'\b\w+\b', mytext)   
    for word in words:
        # 将单词转换为小写，以便统一处理
        word = word.lower()
        # 如果单词在停用词列表中，跳过该单词
        if word in stop_list or len(word) < 2:
            continue        
        word_list.append(word)    
    return " ".join(word_list)
 
data["content_cutted"] = data.raw_text.apply(english_word_cut)
print(data["content_cutted"])

KeyboardInterrupt: 

In [ ]:
def chinese_word_cut(mytext):
    stop_list = []
    try:
        with open(stop_file, encoding='utf-8') as stopword_list:
            stop_list = [line.strip() for line in stopword_list]
    except FileNotFoundError:
        print(f"Error: Stop file '{stop_file}' not found.")
    
    # Use jieba for Chinese word segmentation
    words = jieba.lcut(mytext)
    word_list = []
    
    for word in words:
        # Remove whitespace and filter by length
        word = word.strip()
        if word in stop_list or len(word) < 2:
            continue        
        word_list.append(word)    
    return " ".join(word_list)

# Apply to your data
data["content_cutted"] = data.raw_text.apply(chinese_word_cut)

In [45]:
def chinese_word_cut(mytext):
    jieba.load_userdict(dic_file)
    jieba.initialize()
    try:
        stopwaord_list = open(stop_file, encoding='utf-8')
    except:
        stopwaord_list = []
        print(f"Error: Stop file '{stop_file}' not found.")
    stop_list = []
    flag_list = ['n','nz','vn']

    for line in stopwaord_list:
        line = re.sub(u'\n|\\r','',line)
        stop_list.append(line)

    word_list = []
    #jieba分词
    seg_list = psg.cut(mytext)
    for seg_word in seg_list:
        word = re.sub(u'[^\u4e00-\u9fa5]','',seg_word.word)
        #word = seg_word.word #如果想要分析英语文本，注释这行代码，启动下行代码
        find=0
        for stop_word in stop_list:
            if stop_word == word or len(word) < 2:
                find=1
                break
            if find==0 and seg_word.flag in flag_list:
                word_list.append(word)
    return " ".join(word_list)

In [46]:
data['content_cutted'] = data.raw_text.apply(chinese_word_cut)

In [51]:
data

,raw_text,content_cutted
0,也许可有可无是我的存在//@葵的妈奎的妹坤的妻:[抱抱][抱抱][抱抱][抱抱]哥哥，好想你...,葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵...
1,我讨厌你中间加什么字最甜？我讨厌没你...蔡徐坤//@蔡徐坤我老公鸭://@AK47-KAK...,世界 世界 世界 世界 世界 世界 世界 世界 世界 世界 世界 世界 世界 世界 世界 世...
2,月亮躲起来 连星星 都不在//@葵的妈奎的妹坤的妻:[抱抱][抱抱][抱抱][抱抱]哥哥，好...,月亮 月亮 月亮 月亮 月亮 月亮 月亮 月亮 月亮 月亮 月亮 月亮 月亮 月亮 月亮 月...
3,我只逃避在我封闭的舞台//@葵的妈奎的妹坤的妻:[抱抱][抱抱][抱抱][抱抱]哥哥，好想你...,舞台 舞台 舞台 舞台 舞台 舞台 舞台 舞台 舞台 舞台 舞台 舞台 舞台 舞台 舞台 舞...
4,已不能重来//@葵的妈奎的妹坤的妻:[抱抱][抱抱][抱抱][抱抱]哥哥，好想你呀@蔡徐坤,葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵的 葵...
...,...,...
102308,Oh how'd you'd ever do that to me,
102309,All I do is wait wait wait,
102310,dei tuoi gioielli l’Orrore non è il meno attra...,
102311,"del giorno, delle strade",


### 2 BERTopic Moudel构建

#### 2.1 导入bertopic及相关第三方库

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer

In [ ]:
#### 2.1准备数据

In [53]:
# 使用原始文本而不是预处理文本进行BERTopic分析
from bertopic import BERTopic

# 如果想使用原始文本：
texts = data["raw_text"].tolist()

# 或者如果需要清理但保持文本列表格式：
def preprocess_for_bertopic(text):
    # BERTopic内部处理分词，所以只需基本清理
    import re
    # 保留基本清理但保持中文字符
    text = re.sub(r'[^\w\s\u4e00-\u9fff]', '', str(text))
    return text.strip()

cleaned_texts = [preprocess_for_bertopic(text) for text in data["raw_text"]]

In [54]:
cleaned_texts[:5]  # 查看前5条清理后的文本

['也许可有可无是我的存在葵的妈奎的妹坤的妻抱抱抱抱抱抱抱抱哥哥好想你呀蔡徐坤',
 '我讨厌你中间加什么字最甜我讨厌没你蔡徐坤蔡徐坤我老公鸭AK47KAKAKA东方风云榜让世界看见蔡徐坤\xa0\xa0 蔡徐坤的未完成',
 '月亮躲起来 连星星 都不在葵的妈奎的妹坤的妻抱抱抱抱抱抱抱抱哥哥好想你呀蔡徐坤',
 '我只逃避在我封闭的舞台葵的妈奎的妹坤的妻抱抱抱抱抱抱抱抱哥哥好想你呀蔡徐坤',
 '已不能重来葵的妈奎的妹坤的妻抱抱抱抱抱抱抱抱哥哥好想你呀蔡徐坤']

In [ ]:
#### 2.2 应用BERTopic进行主题建模

In [ ]:

from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

# 加载本地模型
embedding_model = SentenceTransformer('path/to/local/model')
topic_model = BERTopic(embedding_model=embedding_model)
topics, probs = topic_model.fit_transform(cleaned_texts)

# 获取主题信息
topic_info = topic_model.get_topic_info()
print(topic_info)

# 获取每个主题的详细信息
for topic_num in range(topic_info.shape[0]):
    print(f"\n主题 {topic_num}:")
    print(topic_model.get_topic(topic_num))

No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/adapter_config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))"), '(Request ID: fb4d7abb-3fa8-4cd4-9e47-f7f88b57a430)')

In [74]:
from sentence_transformers import SentenceTransformer

#将模型和分词器下载到本地，并指定保存路径
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
cache_dir = r"D:\大学作业文件\大学杂物\A比赛\2025数模\2025数模培训\第五次培训深圳杯2022A\数据\BERTopic\L12_v2"
 
#下载模型
SentenceTransformer.from_pretrained(model_name,cache_dir=cache_dir)
 
print(f"模型分词器已下载到：{cache_dir}")

AttributeError: type object 'SentenceTransformer' has no attribute 'from_pretrained'

In [66]:
# 为了更好的中文支持，可以指定中文嵌入模型
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# 使用中文语言模型
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
# 或者更好的中文支持：
# embedding_model = SentenceTransformer('hfl/chinese-bert-wwm-ext')

topic_model = BERTopic(embedding_model=embedding_model)

# 训练模型
topics, probs = topic_model.fit_transform(cleaned_texts)

# 获取主题信息
topic_info = topic_model.get_topic_info()
print(topic_info)
# 获取每个主题的详细信息
for topic_num in range(topic_info.shape[0]):
    print(f"\n主题 {topic_num}:")
    print(topic_model.get_topic(topic_num))

No sentence-transformers model found with name sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2. Creating a new one with mean pooling.


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/resolve/main/adapter_config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))"), '(Request ID: 91f8b137-30d2-4a63-90ab-e0114262c11b)')

In [ ]:
# 可视化主题
topic_model.visualize_topics()

# 可视化文档分布
topic_model.visualize_documents(cleaned_texts)

# 获取主题频率
topic_freq = topic_model.get_topic_freq()
topic_freq.head(10)

### 2 LDA分析

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [49]:
def print_top_words(model,feature_names, n_top_words):
    tword = []
    for topic_idx,topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        tpoic_w = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        tword.append(tpoic_w)
        print(tpoic_w)
    return tword

In [ ]:
n_features = 1000 #提取1000个特征词语
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df=0.5,
                                min_df=10) #当数据量很小时，min_df可以适当调小
tf = tf_vectorizer.fit_transform(data.content_cutted)

In [67]:
n_topics = 3
lda = LatentDirichletAllocation(n_components = n_topics,
                                max_iter = 50,
                                learning_method = 'batch',
                                learning_offset = 50,
                                #doc_topic_prior = 0.1,
                                #topic_word_prior = 0.01,
                                random_state = 0)
lda.fit(tf)

LatentDirichletAllocation(learning_offset=50, max_iter=50, n_components=3,
                          random_state=0)

#### 2.1输出每个主题对应的主题词

In [79]:
n_top_words=25
tf_feature_names = tf_vectorizer.get_feature_names_out()
topic_word = print_top_words(lda, tf_feature_names, n_top_words)

Topic 0:
梦想 葵葵 老板 奶片 生活 秘密 陪伴 任性 小心 麻麻 小号 人能 时间 自律 星光 星辰 无法 时光 前路 谋面 兔子 生命 葵的 棒棒 妹坤
Topic 1:
世界 健康成长 亲亲 小葵 老板 石头 大家 唯粉 迷妹 姐姐 心心 能量 偶像 故事 人生 样子 霸气 舞台 翅膀 木马 新歌 正宫 经典 小花 坤你是
Topic 2:
世界 奶坤 健健康康 葵花 星球 新家 思思 有缘 舞台 宠物医院 思念 势力 鲜花 晚安 原创 动物 花花世界 初心 内心 菜菜 坤你是 兄弟 活动 义工 全国


#### 2.2输出每篇文章对应主题

In [80]:
import numpy as np

In [81]:
topics = lda.transform(tf)

topic =[]
for t in topics:
    topic.append('Topic #'+str(list(t).index(max(t))))
data['概率最大的主题序号'] = topic
data['每个主题对应概率'] = list(topics)
data.to_excel(r'D:\大学作业文件\大学杂物\A比赛\2025数模\2025数模培训\第五次培训深圳杯2022A\数据\LDAsklearn_origin\data_topic.xlsx',index=False)

#### 2.3可视化

In [82]:
import pyLDAvis
import pyLDAvis.sklearn

In [83]:
pyLDAvis.enable_notebook()
pic=pyLDAvis.sklearn.prepare(lda,tf,tf_vectorizer)
pyLDAvis.display(pic)
PyLDAvis.save_html(pic,r'D:\大学作业文件\大学杂物\A比赛\2025数模\2025数模培训\第五次培训深圳杯2022A\数据\LDAsklearn_origin\caixukun.html')
pyLDAvis.display(pic)

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

In [ ]:
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# 准备 gensim 格式数据
texts = [doc.split() for doc in data["content_cutted"]]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# LDA 模型
lda_model = gensim.models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15)

# 可视化
vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis_data)